In [1]:
import pandas as pd
import numpy as np
import gensim
import requests
from bs4 import BeautifulSoup
import time
import re

In [2]:
df = pd.read_csv('./datasets/df_cleaned.csv')
# df.head()

In [62]:
cities = pd.read_csv('./datasets/cities_clean.csv')
cities.head()

,tweet_count,City,id,tweet_text,timestamp,hashtags,username,mentions,rewtweets,replies,link
0,0,"Paradise, CA",1066843491555205120,"Lord Farquad Quad Squat Squad @Chico, Californ...",2018-11-25 23:58:04+00:00,NaN,areoandmilk,NaN,0,0,https://twitter.com/areoandmilk/status/1066843...
1,1,"Paradise, CA",1066842521601400832,winter edition #queenadailypic released. @Soda...,2018-11-25 23:54:12+00:00,#queenadailypic,leenathequeena,NaN,0,0,https://twitter.com/leenathequeena/status/1066...
2,2,"Paradise, CA",1066841740060098562,Drinking a Def Leppard Pale by @ElysianBrewing...,2018-11-25 23:51:06+00:00,NaN,geradellsworth,@ElysianBrewing @Golden1Center,0,0,https://twitter.com/geradellsworth/status/1066...
3,3,"Paradise, CA",1066841178782482433,"The forgotten. #LimeBike #Reno @Reno, Nevada h...",2018-11-25 23:48:52+00:00,#LimeBike #Reno,alittlegordie,NaN,0,0,https://twitter.com/alittlegordie/status/10668...
4,4,"Paradise, CA",1066839617196961792,SSCC is United and moving Forward !!! The powe...,2018-11-25 23:42:40+00:00,#heritageoffaith,LesSimmons,@sscc7710,1,0,https://twitter.com/LesSimmons/status/10668396...


In [63]:
cities.isna().sum()

tweet_count       0
City              0
id                0
tweet_text        0
timestamp         0
hashtags        882
username          0
mentions       1594
rewtweets         0
replies           0
link              0
dtype: int64

In [16]:
# df = df.drop([2882])
df = df.reset_index(drop=True)

In [5]:
df.shape

(6921, 11)

In [65]:
# counts of hashtages (strip the #)
keyterms = {}
for i in cities['hashtags'].dropna():
    i = i.replace("#", '')
    for j in i.split():
        if j not in keyterms.keys():
            keyterms[j] = 1
        else:
            keyterms[j] += 1


In [7]:
# use hashtag list to build dictionary of keyterms to use combined with that of the gensim model
# Import word vectors into "model."
model = gensim.models.KeyedVectors.load_word2vec_format('../lexvec.enwiki+newscrawl.300d.W.pos.vectors')


In [8]:
keyt = []
keywords = ['fire', 'smoke', 'wildfire', 'campfire', 'forest', 'evacuate', 'hell']

for word in keywords:
    kt = [x[0] for x in model.most_similar(word, topn = 25)]
    keyt+= kt
word_list = list(set(keyt))

In [9]:
important_words = []
for word in keyterms.keys():
    if word in word_list:
        important_words.append(word)

In [66]:
hashtags = list(keyterms.keys())

In [50]:
important_words

['wildfire',
 'evacuated',
 'rescue',
 'fire',
 'fires',
 'firefighters',
 'wildlife',
 'wildfires',
 'displaced',
 'clouds',
 'fuck',
 'evacuees']

In [40]:
hashtags.sort()
len(hashtags)

5996

In [67]:
word_filter = ['fire', 'evac', 'smok', 'burn', 'wild', 'blaz', 'hell', 'department',
              'inferno', 'help']
yet_another_word_list = []
for word in hashtags:
    for wf in word_filter:
        if wf in word.lower():
            yet_another_word_list.append(word)
yet_another_word_list

['CampFire2018',
 'theloveintheairisthickerthanthesmoke',
 'CampFire',
 'campfire',
 'campfireparadise',
 'californiacampfire',
 'firefighter',
 'calfire',
 'campfire2018',
 'californiafires',
 'wildfire',
 'wildfire',
 'californiawildfires',
 'californiawildfires',
 'californiafirefighter',
 'californiafire',
 'evacuees',
 'fire',
 'wildfires',
 'wildfires',
 'wildlandurbaninterface',
 'MetroFire',
 'paradisefirerescue',
 'buttecountyfire',
 'firefighters',
 'cawildfires',
 'cawildfires',
 'woolseyfire',
 'campFire',
 'smoke',
 'Calfire',
 'buttefire',
 'northerncalfire',
 'EarthOnFire',
 'hell',
 'instaFire',
 'forestfire',
 'throughthesmoke',
 'firestick',
 'WildcatFamily',
 'WildcatsRise',
 'calfirefirefighters',
 'evacueetips',
 'smokey',
 'paradisefire',
 'campfirerelief',
 'malibufire',
 'thecampfire',
 'firesmoke',
 'firesmoke',
 'Campfire',
 'CAfire',
 'WildFires',
 'WildFires',
 'smokeinthesky',
 'CampFireDurham',
 'campfireca',
 'CalFire',
 'paradiseonfire',
 'wildlandfire',

In [68]:
# check each tweet for keyterms and score them
keyscore = []
for row in cities.index:
    keyscore.append(0)
    for word in str(cities.iloc[row]['tweet_text']).split():
        if word.lower().replace('#','') in yet_another_word_list:
            try:
                keyscore[row]+=1
            except:
                print(f'fail in row {row}')
cities['key_score'] = keyscore
cities['key_score'].value_counts()


0    895
1    679
2    162
3     37
4     22
5     14
Name: key_score, dtype: int64

In [70]:
df[df['key_score']>0].count()


tweet_count    727
City           727
id             727
tweet_text     727
timestamp      727
hashtags       458
username       727
mentions        67
rewtweets      727
replies        727
link           727
key_score      727
text_nourl     727
dtype: int64

In [71]:
cities[cities['key_score']>0].count()


tweet_count    914
City           914
id             914
tweet_text     914
timestamp      914
hashtags       609
username       914
mentions        93
rewtweets      914
replies        914
link           914
key_score      914
dtype: int64

In [75]:
cities.shape

(1809, 13)

In [78]:
locs = []
delayer = 0
for row in cities.index:
    url = cities.iloc[row]['link']
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'lxml')
    spans = soup.find_all('span', {'class' : 'permalink-tweet-geo-text'})
    try:
        locs.append(spans[0].text.replace("from",'').strip())
    except:
        print(f'tweet #{row} has no location info')
        locs.append('nolocationfound')
    delayer += 1
    if delayer % 25 == 0:
        time.sleep(3)
    if delayer % 75 == 0:
        print(f'Located {delayer} out of 1809 tweets.  {delayer/1809:.2%}')
cities['from_locations'] = locs
cities.to_csv('./datasets/cleanned2.csv', index=False)

Located 75 out of 1809 tweets.  4.15%
Located 150 out of 1809 tweets.  8.29%
Located 225 out of 1809 tweets.  12.44%
Located 300 out of 1809 tweets.  16.58%
tweet #341 has no location info
Located 375 out of 1809 tweets.  20.73%
Located 450 out of 1809 tweets.  24.88%
Located 525 out of 1809 tweets.  29.02%
Located 600 out of 1809 tweets.  33.17%
Located 675 out of 1809 tweets.  37.31%
Located 750 out of 1809 tweets.  41.46%
tweet #801 has no location info
Located 825 out of 1809 tweets.  45.61%
Located 900 out of 1809 tweets.  49.75%
Located 975 out of 1809 tweets.  53.90%
Located 1050 out of 1809 tweets.  58.04%
Located 1125 out of 1809 tweets.  62.19%
Located 1200 out of 1809 tweets.  66.33%
Located 1275 out of 1809 tweets.  70.48%
Located 1350 out of 1809 tweets.  74.63%
Located 1425 out of 1809 tweets.  78.77%
Located 1500 out of 1809 tweets.  82.92%
Located 1575 out of 1809 tweets.  87.06%
Located 1650 out of 1809 tweets.  91.21%
Located 1725 out of 1809 tweets.  95.36%
Located 1

In [80]:
cities['from_locations'].value_counts()

Chico, CA             791
Paradise, CA          576
California, USA       248
Oroville, CA          112
Magalia, CA            17
Gridley, CA            12
Durham, CA             12
South Oroville, CA      9
Sacramento, CA          8
Thermalito, CA          8
Oroville East, CA       3
nolocationfound         2
Folsom, CA              2
Florin, CA              1
Napa, CA                1
Nevada City, CA         1
Redding, CA             1
Yountville, CA          1
Reno, NV                1
Glen Ellen, CA          1
Linda, CA               1
Rocklin, CA             1
Name: from_locations, dtype: int64

In [82]:
cities[cities['from_locations'] == "Napa, CA"]

,tweet_count,City,id,tweet_text,timestamp,hashtags,username,mentions,rewtweets,replies,link,key_score,text_nourl,from_locations
817,4,Butte County,1066840355725668352,Scenes from a successful few days in Napa. Lov...,2018-11-25 23:45:36+00:00,#napavalley #napa #winecountry #california #fall,MereTravels,NaN,0,0,https://twitter.com/MereTravels/status/1066840...,0,Scenes from a successful few days in Napa. Lov...,"Napa, CA"


In [72]:
cities['text_nourl'] = [re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', i) for i in cities['tweet_text']]

In [73]:
cities.to_csv('./datasets/cities_clean2.csv', index=False)

In [74]:
for row in cities.index:
    try:
        cities.iloc[row]['rewtweets']+1
    except:
        print(f'{row} not in df')

In [46]:
df[df['username'] == 'CAL_FIRE']

,tweet_count,City,id,tweet_text,timestamp,hashtags,username,mentions,rewtweets,replies,link,key_score,text_nourl


In [47]:
df.head()

,tweet_count,City,id,tweet_text,timestamp,hashtags,username,mentions,rewtweets,replies,link,key_score,text_nourl
0,6006,Sacramento,1059959242336145408,Vote for our future because they are Hopeful f...,2018-11-07 00:02:31+00:00,NaN,legallylendy,NaN,0,0,https://twitter.com/legallylendy/status/105995...,0,Vote for our future because they are Hopeful f...
1,6005,Sacramento,1059959881237782528,I grew up in a family that celebrated the priv...,2018-11-07 00:05:03+00:00,NaN,jillmansfield_,NaN,0,0,https://twitter.com/jillmansfield_/status/1059...,0,I grew up in a family that celebrated the priv...
2,6004,Sacramento,1059960170313342978,"Getting ready for our ""Bubbles and Bling"" even...",2018-11-07 00:06:12+00:00,NaN,DJsJewelry,NaN,0,0,https://twitter.com/DJsJewelry/status/10599601...,0,"Getting ready for our ""Bubbles and Bling"" even..."
3,901,Butte County,1059960216891080704,VOTE do it for the future . . . . . #pmgridcha...,2018-11-07 00:06:23+00:00,#pmgridchallenge #instatravel #love #wanderlus...,heywoodphotog,NaN,0,0,https://twitter.com/heywoodphotog/status/10599...,0,VOTE do it for the future . . . . . #pmgridcha...
4,6003,Sacramento,1059960413637500928,#ivotedtoday #vote #voteelkgrove #waveofchange...,2018-11-07 00:07:10+00:00,#ivotedtoday #vote #voteelkgrove #waveofchange,karif304,NaN,0,0,https://twitter.com/karif304/status/1059960413...,0,#ivotedtoday #vote #voteelkgrove #waveofchange...


In [51]:
df['City'].value_counts()

Sacramento      5998
Butte County     341
Chico, CA        298
Paradise, CA     284
Name: City, dtype: int64

In [77]:
f'{341/1809:.2%}'

'18.85%'

In [60]:
df[(df['City'] != 'Sacramento') & (df['key_score']>0)]

,tweet_count,City,id,tweet_text,timestamp,hashtags,username,mentions,rewtweets,replies,link,key_score,text_nourl
504,768,"Paradise, CA",1060413731929313280,OMFreakingG! #realfood #chinooksalmon #kingsal...,2018-11-08 06:08:29+00:00,#realfood #chinooksalmon #kingsalmon #fresh #b...,liveg2g,NaN,0,0,https://twitter.com/liveg2g/status/10604137319...,1,OMFreakingG! #realfood #chinooksalmon #kingsal...
559,759,"Chico, CA",1060563632864542720,"Fire in the feather river canyon. @Chico, Cali...",2018-11-08 16:04:09+00:00,NaN,deckinspectah,NaN,0,0,https://twitter.com/deckinspectah/status/10605...,1,"Fire in the feather river canyon. @Chico, Cali..."
575,757,"Chico, CA",1060579478177492992,Closed due to camp fire in #Oroville on Hwy 70...,2018-11-08 17:07:06+00:00,#Oroville #traffic,TotalTrafficSMF,NaN,0,0,https://twitter.com/TotalTrafficSMF/status/106...,1,Closed due to camp fire in #Oroville on Hwy 70...
593,756,"Chico, CA",1060587888654483456,"These are not clouds ladies and gentlemen, THI...",2018-11-08 17:40:32+00:00,NaN,MyLifein3x5s,NaN,0,0,https://twitter.com/MyLifein3x5s/status/106058...,1,"These are not clouds ladies and gentlemen, THI..."
597,755,"Chico, CA",1060588195333791746,Closed due to camp fire in #Oroville on Hwy 70...,2018-11-08 17:41:45+00:00,#Oroville #traffic,TotalTrafficSMF,NaN,1,0,https://twitter.com/TotalTrafficSMF/status/106...,1,Closed due to camp fire in #Oroville on Hwy 70...
602,754,"Chico, CA",1060592862394544128,The view of the Campfire in Feather River Cany...,2018-11-08 18:00:17+00:00,#wildfire #chicoca #paradisecanada #campfire2018,CuratedTravelHL,NaN,0,0,https://twitter.com/CuratedTravelHL/status/106...,3,The view of the Campfire in Feather River Cany...
628,758,"Paradise, CA",1060615959415799808,Camp fire from Bruce Road in Chico. #campfire ...,2018-11-08 19:32:04+00:00,#campfire #Paradise #Chico #wildfire #California,gratefulcake,NaN,1,0,https://twitter.com/gratefulcake/status/106061...,3,Camp fire from Bruce Road in Chico. #campfire ...
664,832,Butte County,1060640915096829952,15 miles away from the Camp Fire in Paradise. ...,2018-11-08 21:11:14+00:00,#campfire,colleenmichelle,NaN,0,0,https://twitter.com/colleenmichelle/status/106...,2,15 miles away from the Camp Fire in Paradise. ...
665,831,Butte County,1060641474054897664,Impending doom. It's kinda scary seeing it thi...,2018-11-08 21:13:27+00:00,#CampFire,TheGreatWhopper,NaN,0,0,https://twitter.com/TheGreatWhopper/status/106...,1,Impending doom. It's kinda scary seeing it thi...
682,749,"Paradise, CA",1060660306660286465,My parents home stands/stood at the point mark...,2018-11-08 22:28:17+00:00,#CalTopo #CampFire #WildFire,kaleb_haugen,NaN,0,2,https://twitter.com/kaleb_haugen/status/106066...,3,My parents home stands/stood at the point mark...


In [59]:
pd.set_option('display.max_rows', None)

In [61]:
df[(df['City'] != 'Sacramento') & (df['key_score']>0)].shape

(474, 13)